# Granite Dells PBR segmentation validation

In [128]:
import laspy
import numpy as np
import os

In [144]:
gt_pc_path = '../../data/granite_dells/associations/semantics.las'  # from ground truth 2D segmentation
#gt_pc_path = '../../data/granite_dells/updated_point_cloud.las'  # from sam2 2D segmentation
assert os.path.exists(gt_pc_path)
gt_pc = laspy.read(gt_pc_path)

# read intensity values of prediction point cloud
prediction_semantics = gt_pc.intensity

# read intensity values of ground truth point cloud
gt_semantics = gt_pc.user_data

In [145]:
# print the number of unique semantics in the prediction point cloud
print('Number of unique semantics in prediction point cloud:', len(np.unique(prediction_semantics)))
# print unique numbers and their counts
print(np.unique(prediction_semantics, return_counts=True))

# print the number of unique semantics in the ground truth point cloud
print('Number of unique semantics in ground truth point cloud:', len(np.unique(gt_semantics)))
# print unique numbers and their counts
print(np.unique(gt_semantics, return_counts=True) )

Number of unique semantics in prediction point cloud: 38
(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37], dtype=uint16), array([     688,     1781,     2065,     2257,     2393,     2425,
           2459,     2527,     2583,     2664,     2799,     2804,
           2861,     3221,     3231,     3277,     3568,     3890,
           4964,     5005,     5259,     5524,     5994,     6115,
           6610,     9126,     9152,    10490,    11267,    12020,
          12050,    14187,    14592,    15651,    17195,    24906,
          26069, 12195635]))
Number of unique semantics in ground truth point cloud: 37
(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36], dtype=uint8), array([12178955,     8469,    12292,     1987,     3558,     7509,
           

In [146]:
# associate the semantics of the prediction point cloud to the ground truth point cloud based on IoU
# create a dictionary to store the mapping
semantics_mapping = {}


N = len(np.unique(prediction_semantics))
# iterate over the unique semantics in the prediction point cloud
for pred_sem in range(0, N-1):
    # get the indices of points in the prediction point cloud with the current semantics
    pred_indices = np.where(prediction_semantics == pred_sem)[0]
    # iterate over the unique semantics in the ground truth point cloud
    for gt_sem in range(1, 37):
        # get the indices of points in the ground truth point cloud with the current semantics
        gt_indices = np.where(gt_semantics == gt_sem)[0]
        # calculate the intersection of the two sets of indices
        intersection = len(np.intersect1d(pred_indices, gt_indices))
        # calculate the union of the two sets of indices
        union = len(np.union1d(pred_indices, gt_indices))
        # calculate the IoU
        iou = intersection / union
        # if the IoU is greater than 0.5, associate the semantics of the prediction point cloud to the ground truth point cloud
        if iou >= 0.5:
            semantics_mapping[(pred_sem, gt_sem)] = iou 
            


print('Semantics mapping:', semantics_mapping)


Semantics mapping: {(1, 3): 0.5878634639696586, (2, 26): 0.8004885993485342, (3, 28): 0.735873850197109, (4, 31): 0.7794068643785405, (5, 25): 0.8903891977760127, (6, 22): 0.9113924050632911, (7, 27): 0.650028522532801, (8, 4): 0.6088551218234215, (9, 35): 0.8474104970455335, (10, 15): 0.8500973393900065, (11, 11): 0.8338308457711443, (12, 29): 0.7501970055161544, (13, 14): 0.8927958833619211, (14, 33): 0.8351553509781358, (15, 23): 0.8904899135446686, (16, 24): 0.8192991631799164, (17, 18): 0.9054726368159204, (18, 9): 0.8967682363804248, (19, 13): 0.8290977315367559, (20, 5): 0.6493993024157085, (21, 30): 0.7162602686139001, (22, 34): 0.895016077170418, (23, 36): 0.8544969833029326, (24, 6): 0.6512032770097286, (25, 1): 0.8027663934426229, (26, 8): 0.7084440969507427, (27, 10): 0.8344421052631579, (28, 12): 0.8290333389177417, (29, 17): 0.8466749727456783, (30, 2): 0.8680070600874837, (31, 20): 0.9600974930362117, (32, 32): 0.8693687879179037, (33, 7): 0.8705472988912479, (34, 21): 0

In [147]:
def validation(iou_threshold):
    TP = 0
    FP = 0
    FN = 0

    for pred_sem in range(0, N-1):
        detection = False
        for gt_sem in range(1, 37):
            if (pred_sem, gt_sem) in semantics_mapping:
                detection = True
                if semantics_mapping[(pred_sem, gt_sem)] >= iou_threshold:
                    TP += 1
                else:
                    FP += 1

        if not detection:
                FP += 1

    # count false negatives
    for gt_sem in range(1, 37):
        missing = True
        for pred_sem in range(0, N-1):
            if (pred_sem, gt_sem) in semantics_mapping:
                if semantics_mapping[(pred_sem, gt_sem)] >= iou_threshold:
                    missing = False
                    break
        if missing:
            FN += 1


    precision = TP / (TP + FP)
    recall = TP / (TP + FN)

    return precision, recall

precision, recall = validation(0.6)
print('Precision:', precision)
print('Recall:', recall)


Precision: 0.9459459459459459
Recall: 0.9722222222222222


In [148]:
def average_validation(iou_list):
    precisions = []
    recalls = []
    for iou in iou_list:
        precision, recall = validation(iou)
        precisions.append(precision)
        recalls.append(recall)

    return precisions, recalls


iou_list = np.arange(0.5, 1.0, 0.05)
print('IoU list:', iou_list)
precisions, recalls = average_validation(iou_list)
print('Precisions:', precisions)
print('Recalls:', recalls)
mean_precisions = np.mean(precisions)
print('Mean precisions:', mean_precisions)
mean_recalls = np.mean(recalls)
print('Mean recalls:', mean_recalls)

IoU list: [0.5  0.55 0.6  0.65 0.7  0.75 0.8  0.85 0.9  0.95]
Precisions: [0.972972972972973, 0.972972972972973, 0.9459459459459459, 0.8918918918918919, 0.8378378378378378, 0.7567567567567568, 0.7027027027027027, 0.43243243243243246, 0.13513513513513514, 0.02702702702702703]
Recalls: [1.0, 1.0, 0.9722222222222222, 0.9166666666666666, 0.8611111111111112, 0.7777777777777778, 0.7222222222222222, 0.4444444444444444, 0.1388888888888889, 0.027777777777777776]
Mean precisions: 0.6675675675675676
Mean recalls: 0.6861111111111111


# Combine filtered point clouds

In [133]:
pc_path = '../../data/granite_dells/associations/semantics.las'
assert os.path.exists(pc_path)
pc = laspy.read(pc_path)
# print the number of points in the point cloud
print('Number of points in the point cloud:', len(pc.points))

Number of points in the point cloud: 12459304


In [134]:
segmented_pc_path = '../../data/granite_dells/semantics_gd.las'
assert os.path.exists(segmented_pc_path)
segmented_pc = laspy.read(segmented_pc_path)

# print the number of points in the segmented point cloud
print('Number of points in the segmented point cloud:', len(segmented_pc.points))
# print unique semantics in the segmented point cloud
print('Unique semantics in the segmented point cloud:', np.unique(segmented_pc.intensity))
# print the number of unique semantics in the segmented point cloud
print('Number of unique semantics in the segmented point cloud:', len(np.unique(segmented_pc.intensity)))

Number of points in the segmented point cloud: 336898
Unique semantics in the segmented point cloud: [155 222 230 232 240 281 284 311 318 323 336 361 364 369 378 418 422 444
 445 450 451 455 458 460 471 480 512 522 536 548 554 560 565 593 618 620
 630 644 650 663 673]
Number of unique semantics in the segmented point cloud: 41


In [135]:
pc.intensity = pc.intensity * 0
print('Unique semantics in the point cloud:', np.unique(pc.intensity))

Unique semantics in the point cloud: [0]


In [136]:
import numpy as np
from sklearn.neighbors import NearestNeighbors

# Combine coordinates
pc_points = np.column_stack((pc.x, pc.y, pc.z))
segmented_points = np.column_stack((segmented_pc.x, segmented_pc.y, segmented_pc.z))

# Build nearest neighbor index
nn = NearestNeighbors(n_neighbors=1, algorithm='kd_tree')
nn.fit(pc_points)

# Find nearest neighbors with distance threshold
distances, indices = nn.kneighbors(segmented_points)
epsilon = 1e-3  # Adjust based on your data precision

# Create masks and flatten arrays
valid_matches = distances.ravel() < epsilon
valid_indices = indices.ravel()[valid_matches]
valid_intensity = segmented_pc.intensity[valid_matches]

# Safe assignment
if len(valid_indices) > 0:
    pc.intensity[valid_indices] = valid_intensity
else:
    print(f"No matches found within {epsilon} distance")

In [137]:
# re-arrange pc.intensity to start from 0, 1, 2, 3, ...

# get unique semantics in the point cloud
unique_semantics, counts = np.unique(pc.intensity, return_counts=True)
# create a dictionary to store the mapping
mapping = {}
# iterate over the unique semantics in the point cloud
for i, s in enumerate(unique_semantics):
    # exclude the semantics 0
    if s != 0:
        # map the semantics to the new values
        mapping[s] = i-1

mapping[0] = len(unique_semantics) - 1

# map the semantics to the point cloud
pc.intensity = np.vectorize(mapping.get)(pc.intensity)

In [138]:
# Create a new LAS file
header = laspy.LasHeader(version="1.4", point_format=3)  # Use version 1.4 and format 3 for intensity support

# Create a new LAS object using your existing data
las = laspy.LasData(header)

# Populate coordinates
las.x = pc.x
las.y = pc.y
las.z = pc.z

# Add intensity (if not already present)
if hasattr(pc, "intensity"):
    las.intensity = pc.intensity
    las.user_data = pc.user_data
else:
    # Create new intensity field (scaled to 0-65535)
    las.add_extra_dim(laspy.ExtraBytesParams(name="intensity", type=np.uint16))
    las.intensity = (pc.intensity * 65535).astype(np.uint16)  # Normalize if needed

# Save to file
updated_pc_path = '../../data/granite_dells/updated_point_cloud.las'
las.write(updated_pc_path)